This notebook attempts to speed up the prediction_vs_solution plot with julia.
There are 3 julia methods.

1. Is loading julia from a system image and calling julia from python. You will need to run the `tools/make_sysimage.jl` file. This will take a while to run. After, try running the `tools/test_sysimage.py` file. This will make sure everything is working properly for maestroflame to run julia. 

2. Takes a more naive approach, outputting files to disk and then using `os.system()` commands to call a julia script `maestroflame/pred_v_sol.jl`. This is slow because of the start up time. 

3. A third incomplete method that I had trouble getting to work was loading julia fron a virtual enviroment. Within the `pred_v_sol.jl` file you can see the load env commands. This might get around the julia start up time?

## Results:

In [1]:
!pip install ../

Processing /Users/chris/Desktop/temp/ml-reactions/maestroflame


  Created wheel for MaestroFlame: filename=MaestroFlame-0.1.0-py3-none-any.whl size=28531 sha256=0a411bbb250fe1e3761e58587637eedb752d622392253476ddbf476399689040
  Stored in directory: /private/var/folders/j8/2s1d11vs6xx9g1f18gn0y4rm0000gn/T/pip-ephem-wheel-cache-0r7mzeag/wheels/bc/1a/ae/3d3ed493ea0c7ad32bf5ff3d67158953fe0de6520f90944766
Successfully built MaestroFlame
  Attempting uninstall: MaestroFlame
    Found existing installation: MaestroFlame 0.1.0
    Uninstalling MaestroFlame-0.1.0:
      Successfully uninstalled MaestroFlame-0.1.0


In [2]:
import maestroflame

In [3]:
#this is the path to your data files
data_path = '../../data/data1/flame/'

#These is the input/output prefix of your datafile names.
input_prefix = 'react_inputs_*'
output_prefix = 'react_outputs_*'

#Plotfile prefixes, used for visualization purposes.
plotfile_prefix = 'flame_*'

#By default, this package will save your model, logs of the training and testing data during training,
#and plots to a directory. Here you specify that directory.
output_dir = '/Users/chris/Desktop/temp/ml-reactions/maestroflame/examples/testing123/'

#The log file. Everything that is printed during training also goes into this file in case something
#gets interrupted.
log_file = output_dir + "log.txt"

In [4]:
#We first remove the directory we just generated. If you don't do this you'll get an error. This is to protect
#this package from overwriting your data in case one forgets to change the output_dir when training a new model

!rm -r testing123/

In [5]:
from maestroflame.train import NuclearReactionML
nrml = NuclearReactionML(data_path, input_prefix, output_prefix, plotfile_prefix,
                output_dir, log_file, DEBUG_MODE=True, DO_PLOTTING=True,
                SAVE_MODEL=True, DO_HYPER_OPTIMIZATION=False)

Model starting on : 12/16/2021 14:03:57
input_prefix react_inputs_*
output_prefix react_outputs_*
output_dir /Users/chris/Desktop/temp/ml-reactions/maestroflame/examples/testing123/
DEBUG_MODE True
DO_PLOTTING True
DO_HYPER_OPTIMIZATION False
Loading Input Files...
Loading Output Files...
Loaded data successfully!


In [6]:
#the package provides preset up loss functions and networks to use
from maestroflame.losses import log_loss
from maestroflame.networks import Net
import torch.optim as optim
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_epochs = 1
model = Net(16, 16, 16, 16, 14)

# get model to cuda if possible
model.to(device=device)
optimizer = optim.Adam(model.parameters(), lr=1e-5)

nrml.train(model, optimizer, num_epochs, log_loss, save_every_N=10)

Cost at epoch 0 is 32.22436998435723
Saving...


In [7]:
from maestroflame.plotting import plotting_standard

import time

plot_class = plotting_standard(nrml.model, nrml.fields, nrml.test_loader, nrml.cost_per_epoc, nrml.component_losses_test,
                        nrml.component_losses_train, nrml.cost_per_epoc_test, nrml.output_dir)

plot_class.do_prediction_vs_solution_plot(use_julia=True)

start_time = time.time()
plot_class.do_prediction_vs_solution_plot(use_julia=False)
print("%s seconds total for python " % (time.time() - start_time))

--- Internal Timing Julia
8.286669969558716 seconds to start julia 


/Users/chris/anaconda3/lib/python3.8/site-packages/maestroflame/plotting.py:115: RuntimeWarning: invalid value encountered in log10
  pred_plot = np.log10(pred_plot)


29.997073888778687 seconds to do preprocessing 
10.120919942855835 seconds to do plot 
48.40560579299927 seconds total for first julia plotting method 
0.33952999114990234 seconds to save data to file 
46.47067403793335 seconds to run julia pred_v_sol.jl 
46.81110095977783 seconds total to run julia method 2 
6.319821119308472 seconds total for python 
